# Importing packages

In [250]:
# !pip install mne

In [251]:
# !pip install pyxdf

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mne
import pyxdf
import glob
import os
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy
import sklearn
# ------------------------------------------------------------------------------------------
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC

# ------------------------------------------------------------------------------------------
from mne.decoding import CSP
from mne import Epochs, pick_types
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf

In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

In [3]:
from FBCSP_V7 import FBCSP_V4 as FBCSP 

In [4]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [5]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

# Data Loading

In [257]:
# from google.colab import drive
# drive.mount('/content/drive')

In [6]:
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S002\eeg"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S002\eeg\Remaining Data"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S002\s_2_all"

# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S003\s3"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S003\s3_45"
# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\ses-S003\s3_00"

# Anwar_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Anwar\s-00" # ALL ANWAR


# sherif_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\ses-S002\eeg"
# sherif_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\ses-S003\eeg"

sherif_data_path = "E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\s-00" #ALL SHERIF

# magdy_data_path = "E:\BCI\A-Our Motor Data\magdy\sub-magdy\ses-S001\eeg"

# saad_data_path = "E:\BCI\A-Our Motor Data\omar\sub-omar\ses-S001\eeg"

folder_path = sherif_data_path
files   = glob.glob(folder_path + '/*eeg.xdf')
# files   = glob.glob(folder_path)
len(files)     # if  return zero,then no file is loaded

225

In [7]:
files

['E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-001_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-006_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-010_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-013_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-014_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-019_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-020_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-022_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Baseline_run-027_eeg.xdf',
 'E:\\BCI\

In [8]:
# Filter the paths
filtered_paths = [path for path in files if ('Right Grasp' in path or 'Right Release' in path) and 
                                            ('B_Right Grasp' not in path and 'B_Right Release' not in path)]

# filtered_paths = [path for path in files if ('Right Grasp' in path or 'Right Release' in path or 'Baseline' in path) and 
#                                             ('B_Right Grasp' not in path and 'B_Right Release' not in path and 'B_Baseline' not in path)]

# Print the number of filtered paths and the paths themselves
print(f'Number of filtered paths: {len(filtered_paths)}')
filtered_paths

Number of filtered paths: 150


['E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-003_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-007_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-008_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-011_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-012_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-016_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-017_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-023_eeg.xdf',
 'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_r

In [9]:
all_files = filtered_paths
len(all_files)

150

# from XDF to MNE

In [12]:
one_file = all_files[0]
one_file

'E:\\BCI\\A-Our Motor Data\\Good Data\\sub-Sherif\\s-00\\sub-sherif_ses-S002_task-Right Grasp_run-003_eeg.xdf'

In [17]:
filename = one_file.split('/')[-1]
# Extract the task name part
task_name = filename.split('_')[-3]
# Replace the hyphen with a space to match the desired format
task_name = task_name.split('-')[-1]

print(task_name)

Right Grasp


In [13]:
# read stream from xdf
streams, header = pyxdf.load_xdf(one_file)
# extract data
data = streams[0]["time_series"].T
# check that it is 9 channels
# assert data.shape[0] == 9 # 9 raw EEG channels

#get channels count
ch_count = int(streams[0]["info"]["channel_count"][0])
# extract channels names
# ch_names = []
# for i in range(ch_count):
#   ch_names.append(streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0])
# extract sampling ratw
fs = float(streams[0]["info"]["nominal_srate"][0])
# create info
# info = mne.create_info(ch_names, fs, "eeg")

In [14]:
# extract channels names
ch_names = []
for i in range(ch_count):
  ch_name = streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0]
  # ch_name= ch_name.split("\n")[1]
  ch_name= ch_name.split("\n")[0]
  ch_names.append(ch_name)

In [15]:
ch_names

['C3', 'CZ', 'C4', 'FCZ', 'FC3', 'CP4', 'CPZ', 'CP3', 'FC4']

In [18]:
info = mne.create_info(ch_names, fs, "eeg")

In [20]:
event_id = {'Right Grasp': 1, 'Right Release': 2, 'Baseline': 3}

In [36]:
task_number = [event_id[task_name] ]
print(task_number)

[1]


In [21]:
# crop parametes
t_start = 2
t_end = 3.5
# t_end = 6

idx_start = int(fs*t_start)
idx_end = int(fs*t_end)

In [31]:
# crop file
streams, header = pyxdf.load_xdf(one_file)
data = streams[0]["time_series"].T
print(data.shape)
cropped_file = data[:,idx_start:idx_end+1]
print(one_file)
print(cropped_file.shape)

(9, 3525)
E:\BCI\A-Our Motor Data\Good Data\sub-Sherif\s-00\sub-sherif_ses-S002_task-Right Grasp_run-003_eeg.xdf
(9, 751)


In [32]:
# convert combined_array to size (1,9, 751)
eeg_data = np.expand_dims(cropped_file, axis=0)
print(eeg_data.shape)
eeg_data

(1, 9, 751)


array([[[-12.234518  , -12.334176  , -12.396158  , ...,   0.08418522,
           0.24656044,   0.14209579],
        [-13.204489  , -13.961244  , -14.867891  , ...,  -0.7805864 ,
          -1.4006026 ,  -2.2010431 ],
        [-11.828105  , -13.041146  , -14.36721   , ...,   5.2913985 ,
           5.6788297 ,   5.7588573 ],
        ...,
        [-19.253828  , -20.00418   , -20.730011  , ...,   6.411288  ,
           5.388779  ,   3.9650872 ],
        [-15.983969  , -16.430798  , -16.661638  , ...,   5.0888996 ,
           4.713101  ,   3.943438  ],
        [ -8.81132   ,  -9.616292  , -10.705974  , ...,   0.14039935,
           0.37618965,   0.5503985 ]]], dtype=float32)

In [33]:
epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=0)

In [37]:
# epochs.set_montage('standard_1020')
epochs.filter(1., None)
# epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

epochs.event_id = event_id
epochs.events[:,2] = task_number

In [38]:
epochs

Number of events,1
Events,Baseline: 0Right Grasp: 1Right Release: 0
Time range,0.000 – 1.500 s
Baseline,off


In [39]:
epochs.filter(7.0, 32.0)

Number of events,1
Events,Baseline: 0Right Grasp: 1Right Release: 0
Time range,0.000 – 1.500 s
Baseline,off


## Get Epochs Data

In [40]:
data, labels = epochs.get_data(), epochs.events[:,-1]

In [41]:
print(data.shape)
print(labels.shape)
labels

(1, 9, 751)
(1,)


array([1])

## Trial Prediction Time

## Load model

In [59]:
import pickle

# Load the FBCSP from the file
with open('fbcsp_clf_sherif.pkl', 'rb') as file:
    loaded_fbcsp_clf = pickle.load(file)

In [60]:
# Load the trained model from the file
with open('LDA_clf_sherif.pkl', 'rb') as file:
    loaded_LDA_clf = pickle.load(file)

## Trial Prediction with loaded models

In [64]:
trial_features = loaded_fbcsp_clf.extractTrialFeatures(data)
# print(trial_features.shape)
trial_features

(1, 8)


array([[-0.98284818, -0.24993581,  0.24228632, -0.21664909, -0.98323606,
        -1.37446386, -0.09868319, -1.60380917]])

In [66]:
prediction = loaded_LDA_clf.predict(trial_features)
prediction[0]

1.0